In [1]:
using BenchmarkTools, StructArrays, LinearAlgebra, Profile, PProf, Zygote, Zygote.ChainRulesCore

In [2]:
struct Variable{D}
    d :: D    
end
mean(v::Variable) = mean(v.d)
var(v::Variable) = var(v.d)
mean_var(x) = mean(x), var(x)
struct Normal{T}
    m :: T
    v :: T
end
mean(n::Normal) = n.m
var(n::Normal) = n.v
Base.zero(::Variable{<:Normal{T}}) where { T } = Variable(Normal(zero(T), zero(T)))
Base.zero(::Type{Variable{T}}) where { T } = Variable(zero(T))
Base.zero(::Type{Normal{T}}) where { T } = Normal(zero(T), zero(T))
function Base.:*(n1::Variable{<:Normal}, n2::Variable{<:Normal})
    m1, v1 = mean_var(n1)
    m2, v2 = mean_var(n2)

    return Variable(Normal(m1 * m2, v1 * v2 + v2 * m1 * m1 + v1 * m2 * m2))
end
function Base.:+(n1::Variable{<:Normal}, n2::Variable{<:Normal})
    m1, v1 = mean_var(n1)
    m2, v2 = mean_var(n2)

    return Variable(Normal(m1 + m2, v1 + v2))
end
LinearAlgebra.dot(n1::Variable{<:Normal}, n2::Variable{<:Normal}) = n1 * n2
;

In [3]:
dimin = 1000
dimout = 500
T = Float32

W = Variable.(Normal.(randn(T,dimout,dimin), abs.(randn(T,dimout,dimin))))
b = Variable.(Normal.(randn(T,dimout), abs.(randn(T,dimout))))
x = Variable.(Normal.(randn(T,dimin), abs.(randn(T,dimin))))

W_sa = StructArray(W; unwrap = t -> t <: Normal)
b_sa = StructArray(b; unwrap = t -> t <: Normal)
x_sa = StructArray(x; unwrap = t -> t <: Normal)
;

In [4]:
function Base.:*(A::Matrix{T1}, x::Vector{T2}) where { T1 <: Variable, T2 <: Variable }
    y = zeros(T1, size(A, 1))
    for i = 1:size(A, 1), k = 1:size(A, 2)
        y[i] += A[i,k] * x[k]
    end
    return y
end

In [5]:
# function ChainRulesCore.rrule(::typeof(*), A::Matrix{T1}, x::Vector{T2}) where { T1 <: Variable, T2 <: Variable }
#     A * x, Δ -> (NoTangent(), NoTangent(), NoTangent())
# end

In [6]:
@benchmark $W * $x + $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  363.000 μs … 635.042 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     370.979 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   381.257 μs ±  20.072 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▆▆▅▃▃▃▂▂▂▂▂▂▃▂▂▂▁▁▂▃▅▅▄▄▃▃▂▁▁▁▁▁          ▁▁▁▂▂▁▁▁          ▂
  █████████████████████████████████████████████████████▇▇▇▇▇▆▆▇ █
  363 μs        Histogram: log(frequency) by time        439 μs <

 Memory estimate: 8.25 KiB, allocs estimate: 2.

In [7]:
@benchmark $W_sa * $x_sa + $b_sa

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  82.417 μs … 118.792 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     89.708 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   90.497 μs ±   3.086 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▁             ▆█▇█▇▇▆▄▃▄▃▄▃▂▂▂▁▁▁   ▁▁▁                    ▃
  ▇██▇▅▁▁▃▃▅▃▃▃▄▁▇█████████████████████████████▇▇▆▇▇█▇█▇▇▇▆▇▇▇ █
  82.4 μs       Histogram: log(frequency) by time       105 μs <

 Memory estimate: 8.73 KiB, allocs estimate: 16.

In [8]:
abstract type Layer end
struct Dense{T1, T2} <: Layer
    W :: T1
    b :: T2
end
(l::Dense)(x) = l.W * x + l.b

In [9]:
layer = Dense(W, b)
layer_sa = Dense(W_sa, b_sa);

In [10]:
@benchmark layer($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  363.458 μs … 649.375 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     393.667 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   399.091 μs ±  22.707 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆▅▄▄▂▁▂▁▂▂▂▂▂▁▁▁▁▁▅█▅▆▄▃▃▂▂▂▁▁▁    ▁▂▃▄▅▆▆▅▄▃▂▂▂▁▁▁           ▂
  ██████████████████████████████████████████████████████▇▇▆▇▅▆▆ █
  363 μs        Histogram: log(frequency) by time        451 μs <

 Memory estimate: 8.25 KiB, allocs estimate: 2.

In [11]:
@benchmark layer_sa($x_sa)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  82.750 μs … 118.416 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     89.792 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   90.912 μs ±   3.536 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▆███▇▇▅▄▄▃▄▃▂▂▁▂▁▁  ▁▁▁▁        ▁▁   ▁         ▃
  ▇▅▃▁▁▁▃▁▃▁▁▁▁▃██████████████████████████▇▇███████▆███▇█▇▇▇▆▇ █
  82.8 μs       Histogram: log(frequency) by time       107 μs <

 Memory estimate: 8.78 KiB, allocs estimate: 18.

In [12]:
dlayer = gradient(l -> sum(mean.(l(x))), layer)

ErrorException: Mutating arrays is not supported -- called setindex!(Vector{Variable{Normal{Float32}}}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [16]:
dlayer_sa = gradient(l -> sum(mean.(l(x_sa))), layer_sa)

ErrorException: Mutating arrays is not supported -- called setindex!(StructVector{Variable{Normal{Float32}}, @NamedTuple{d::StructVector{Normal{Float32}, @NamedTuple{m::Vector{Float32}, v::Vector{Float32}}, Int64}}, Int64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations
